# Visualization and results

The following notebook is used to visualize the outputs from the deep learning models as well as the comparison methods. Additionally, there is functionality for generating result metrics that can also be found in the papers that come out of this work.


Author: Akshay Aravamudan
Last edited: January 1st 2023

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from dataset_classes import *
from Interpolation import *
from Downscaler import *
from Distance_Ranking import *
from RDN import *
from RCAN import *
import torch
import sys
from torch.utils.data.dataloader import DataLoader
import numpy as np
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

## Utility functions

In [ ]:
def load_pytorch_models(model_name, input_type):
    config = {}
    config['run_dir'] = 'runs/run_final'
    config['model_type'] = model_name
    config['topo_inclusion'] = input_type

    device = f"cuda:0"
    device = torch.device(device if torch.cuda.is_available() else "cpu")


    model_dict = np.load(config['run_dir'] + '/best_model'+config['model_type']+config['topo_inclusion']+'.npy', allow_pickle=True).item()
    if model_name == 'RDN':
      model = RDN(scale_factor=8,
                  num_channels=1,
                  num_features=model_dict['num_features'],
                  growth_rate=model_dict['num_features'],
                  num_blocks=model_dict['num_blocks'],
                  num_layers=model_dict['num_layers'],
                  topo_inclusion=config['topo_inclusion'],
                  dropout_prob_input=model_dict['dropout_prob_input'],
                  dropout_prob_topo_1=model_dict['dropout_prob_topo_1'],
                  dropout_prob_topo_2=model_dict['dropout_prob_topo_2']
                  ).to(device)
    else:
      model = RCAN(scale=8, 
                  num_features=model_dict['num_features'], 
                  num_rg=model_dict['num_rg'], 
                  num_rcab=model_dict['num_rcab'], 
                  reduction=model_dict['reduction'],
                  topo_inclusion=config['topo_inclusion'],
                  dropout_prob_input=model_dict['dropout_prob_input'],
                  dropout_prob_topo_1=model_dict['dropout_prob_topo_1'],
                  dropout_prob_topo_2=model_dict['dropout_prob_topo_2']).to(device)

    model.load_state_dict(torch.load(config['run_dir']+'/best_model'+config['model_type']+config['topo_inclusion']+'.h5',map_location=torch.device('cpu')))
    # model.eval()
    
    return model, device